In [15]:
!pip -q uninstall -y mediapipe protobuf opencv-python opencv-python-headless
!pip -q install "protobuf<5" opencv-python-headless pillow numpy tqdm
!pip -q install "mediapipe==0.10.14"

!python -c "import mediapipe as mp; print('mediapipe', mp.__version__, 'solutions:', hasattr(mp,'solutions')); print('pose:', hasattr(mp.solutions,'pose'))"


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
2025-12-19 21:03:14.912713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766178194.953859    1390 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766178194.962302    1390 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 21:03:14.985251: I tensorflow/core/platform/cp

In [38]:
%cd /content


/content


In [39]:
%%bash
set -e
cd /content

# now it's safe to delete/recreate
rm -rf tp
mkdir -p tp

# unzip again (adjust zip name if needed)
unzip -o TP_solution_code_v2.zip -d /content/tp

ls -lah /content/tp | head


Archive:  TP_solution_code_v2.zip
  inflating: /content/tp/GenVanillaNN.py  
  inflating: /content/tp/Skeleton.py  
  inflating: /content/tp/GenNearest.py  
  inflating: /content/tp/GenGAN.py   
  inflating: /content/tp/DanceDemo.py  
  inflating: /content/tp/README.txt  
total 52K
drwxr-xr-x 2 root root 4.0K Dec 19 21:23 .
drwxr-xr-x 1 root root 4.0K Dec 19 21:23 ..
-rw-r--r-- 1 root root 3.4K Dec 19 01:42 DanceDemo.py
-rw-r--r-- 1 root root 6.8K Dec 19 01:38 GenGAN.py
-rw-r--r-- 1 root root 1.5K Dec 19 01:37 GenNearest.py
-rw-r--r-- 1 root root  11K Dec 19 01:37 GenVanillaNN.py
-rw-r--r-- 1 root root  822 Dec 19 01:38 README.txt
-rw-r--r-- 1 root root  11K Dec 19 01:37 Skeleton.py


In [40]:
!mv /content/VideoSkeleton.py /content/VideoReader.py /content/Vec3.py /content/tp/ 2>/dev/null || true
!ls -la /content/tp | grep -E "VideoSkeleton.py|VideoReader.py|Vec3.py"


-rw-r--r-- 1 root root  3595 Dec 19 21:23 Vec3.py
-rw-r--r-- 1 root root  2277 Dec 19 21:23 VideoReader.py
-rw-r--r-- 1 root root 10353 Dec 19 21:23 VideoSkeleton.py


In [41]:
!mkdir -p /content/tp/data
!mv /content/*.mp4 /content/tp/data/ 2>/dev/null || true
!ls -la /content/tp/data


total 8
drwxr-xr-x 2 root root 4096 Dec 19 21:23 .
drwxr-xr-x 3 root root 4096 Dec 19 21:23 ..


In [42]:
%cd /content/tp

# delete the tiny dataset
!rm -f data/karate1.pkl
!rm -rf data/karate1

# recompute keeping 1 frame out of 1 (all frames)
!python VideoSkeleton.py data/karate1.mp4 true 1


/content/tp
2025-12-19 21:24:00.428819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766179440.473158    6462 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766179440.486141    6462 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 21:24:00.541331: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Current Working Directory:  /content/tp
Filename= data/karate1.mp4
directory: data/karate1
create directory: data

In [43]:
!ls data/karate1 | wc -l


0


In [44]:
%%bash
cd /content/tp
python - <<'PY'
import re
p="GenVanillaNN.py"
s=open(p,"r",encoding="utf-8").read()
s=re.sub(r'optSkeOrImage\s*=\s*\d+', 'optSkeOrImage = 1', s, count=1)
open(p,"w",encoding="utf-8").write(s)
print("Set optSkeOrImage = 1")
PY


Set optSkeOrImage = 1


In [46]:
%%bash
set -e
cd /content/tp

python - <<'PY'
import re, pathlib

p = pathlib.Path("DanceDemo.py")
s = p.read_text(encoding="utf-8")

# 0) Ensure imports exist
if "import os" not in s:
    s = s.replace("import sys\n", "import sys\nimport os\n")

# 1) Add output dir + writer init at start of draw()
# Insert right after: def draw(self):
s = re.sub(
    r"def draw\(self\):\n",
    "def draw(self):\n"
    "        os.makedirs('data/out', exist_ok=True)\n"
    "        writer = None\n",
    s,
    count=1
)

# 2) Replace the first cv2.imshow(...) on 'vis' with: save JPG + write MP4
# This targets the exact pattern in your traceback: cv2.imshow(..., vis)
s, n = re.subn(
    r"\s*cv2\.imshow\([^\n]*vis\)\s*\n",
    "            # Colab: no cv2.imshow(). Save previews + write video.\n"
    "            if i % 10 == 0:\n"
    "                cv2.imwrite(f\"data/out/vis_{i:04d}.jpg\", vis)\n"
    "            if writer is None:\n"
    "                h, w = vis.shape[:2]\n"
    "                writer = cv2.VideoWriter(\n"
    "                    \"data/out/result.mp4\",\n"
    "                    cv2.VideoWriter_fourcc(*\"mp4v\"),\n"
    "                    25,\n"
    "                    (w, h)\n"
    "                )\n"
    "            writer.write(vis)\n",
    s,
    count=1
)

if n == 0:
    print("WARNING: No cv2.imshow(...vis) found to replace. Check DanceDemo.py formatting.")

# 3) Remove waitKey / destroyAllWindows (they also depend on GUI)
s = re.sub(r"\s*key\s*=\s*cv2\.waitKey\([^\n]*\)\s*&\s*0xFF\s*\n", "", s)
s = re.sub(r"\s*cv2\.destroyAllWindows\(\)\s*\n", "", s)

# 4) Ensure writer.release() happens before returning from draw()
# Add near end of draw(): just before the function ends (simple heuristic)
if "writer.release()" not in s:
    s = re.sub(
        r"(\n\s*print\([^\n]*\)\s*\n\s*$)",
        "\n        if writer is not None:\n"
        "            writer.release()\n"
        "        print('Saved previews to data/out/ and video to data/out/result.mp4')\n"
        + r"\1",
        s,
        count=1
    )

p.write_text(s, encoding="utf-8")
print("✅ Patched DanceDemo.py to save previews + MP4 (no GUI).")
PY


✅ Patched DanceDemo.py to save previews + MP4 (no GUI).


In [47]:
%%bash
cd /content/tp
python - <<'PY'
from pathlib import Path
p = Path("DanceDemo.py")
s = p.read_text(encoding="utf-8")

if "breakdef _usage" in s:
    s = s.replace("breakdef _usage", "break\n\ndef _usage")
    p.write_text(s, encoding="utf-8")
    print("✅ Fixed: replaced 'breakdef _usage' with 'break\\n\\ndef _usage'")
else:
    print("Didn't find 'breakdef _usage'. Showing suspicious lines:")
    lines = s.splitlines()
    for i, line in enumerate(lines, 1):
        if "def _usage" in line or "breakdef" in line:
            print(i, line)
PY


✅ Fixed: replaced 'breakdef _usage' with 'break\n\ndef _usage'


In [48]:
%%bash
cd /content/tp
python - <<'PY'
from pathlib import Path
import re

p = Path("DanceDemo.py")
s = p.read_text(encoding="utf-8")

# Insert conversion right before first writer.write(...) if not already present
if "vis_u8" not in s:
    s = re.sub(
        r"(writer\.write\(vis\))",
        "# make sure vis is uint8 BGR for OpenCV writer\n"
        "            if vis.dtype != np.uint8:\n"
        "                vis_u8 = np.clip(vis, 0, 1)\n"
        "                vis_u8 = (vis_u8 * 255.0).astype(np.uint8)\n"
        "            else:\n"
        "                vis_u8 = vis\n"
        "\n"
        "            " + r"\1".replace("vis", "vis_u8"),
        s,
        count=1
    )

# Replace imwrite(..., vis) -> imwrite(..., vis_u8) if present
s = s.replace("cv2.imwrite(f\"data/out/vis_{i:04d}.jpg\", vis)", "cv2.imwrite(f\"data/out/vis_{i:04d}.jpg\", vis_u8)")
# Replace any remaining writer.write(vis) -> writer.write(vis_u8)
s = s.replace("writer.write(vis)", "writer.write(vis_u8)")

p.write_text(s, encoding="utf-8")
print("✅ Patched DanceDemo.py to convert vis to uint8 before imwrite/video write.")
PY


✅ Patched DanceDemo.py to convert vis to uint8 before imwrite/video write.


In [49]:
%%bash
cd /content/tp
python - <<'PY'
from pathlib import Path
import re

p = Path("DanceDemo.py")
s = p.read_text(encoding="utf-8")

# helper expression: convert vis -> uint8 safely (handles float [0,1] or already uint8)
conv = "(vis if vis.dtype==np.uint8 else (np.clip(vis,0,1)*255.0).astype(np.uint8))"

# 1) Replace any imwrite that uses vis_u8 -> use inline conversion from vis
s = re.sub(
    r'cv2\.imwrite\(([^,]+),\s*vis_u8\s*\)',
    r'cv2.imwrite(\1, ' + conv + r')',
    s
)

# 2) Replace any writer.write(vis_u8) -> writer.write(inline conversion)
s = re.sub(
    r'writer\.write\(\s*vis_u8\s*\)',
    'writer.write(' + conv + ')',
    s
)

# 3) (Optional) If you still have a vis_u8 conversion block, leave it—harmless.
p.write_text(s, encoding="utf-8")
print("✅ Patched: writes now convert 'vis' to uint8 inline (no vis_u8 variable needed).")
PY


✅ Patched: writes now convert 'vis' to uint8 inline (no vis_u8 variable needed).


In [50]:
%%bash
cd /content/tp
python - <<'PY'
from pathlib import Path
import re

p = Path("DanceDemo.py")
s = p.read_text(encoding="utf-8")

# Remove lines that call cv2.waitKey(...)
s = re.sub(r'^\s*.*cv2\.waitKey\(.*\).*\n', '', s, flags=re.MULTILINE)

# Remove common "press q to quit" patterns (may leave an empty if-block otherwise)
s = re.sub(r'^\s*if\s+\(?.*ord\(\'q\'\).*\)?:\s*\n\s*break\s*\n', '', s, flags=re.MULTILINE)

p.write_text(s, encoding="utf-8")
print("✅ Removed cv2.waitKey() lines from DanceDemo.py")
PY


✅ Removed cv2.waitKey() lines from DanceDemo.py


In [57]:
%%bash
set -e
cd /content/tp

python - <<'PY'
from pathlib import Path
import re

p = Path("DanceDemo.py")
s = p.read_text(encoding="utf-8")

# ensure import os
if re.search(r'^\s*import os\s*$', s, flags=re.M) is None:
    s = s.replace("import sys\n", "import sys\nimport os\n")

# ensure numpy import
if re.search(r'^\s*import numpy as np\s*$', s, flags=re.M) is None:
    s = s.replace("import cv2\n", "import cv2\nimport numpy as np\n")

# patch draw signature safely (your draw has args, so match any def draw(self,...): )
s = re.sub(
    r"(def\s+draw\s*\(\s*self[^)]*\)\s*:\s*\n)",
    r"\1        os.makedirs('data/out', exist_ok=True)\n",
    s,
    count=1
)

# disable GUI calls
s = re.sub(r"^(\s*)cv2\.imshow\(.*\)\s*$", r"\1# \g<0>  # disabled on Colab", s, flags=re.M)
s = re.sub(r"^(\s*)cv2\.waitKey\(.*\)\s*$", r"\1# \g<0>  # disabled on Colab", s, flags=re.M)
s = re.sub(r"^(\s*)cv2\.destroyAllWindows\(\)\s*$", r"\1# \g<0>  # disabled on Colab", s, flags=re.M)

p.write_text(s, encoding="utf-8")
print("✅ Patched DanceDemo.py in-place (no self-copy).")
PY


✅ Patched DanceDemo.py in-place (no self-copy).


In [60]:
%%bash
set -e
cd /content/tp

python - <<'PY'
import re
from pathlib import Path

p = Path("DanceDemo.py")
s = p.read_text(encoding="utf-8")

# Ensure imports exist
if "import os" not in s:
    s = s.replace("import sys\n", "import sys\nimport os\n")
if "import numpy as np" not in s:
    s = s.replace("import cv2\n", "import cv2\nimport numpy as np\n")

# 1) Disable GUI safely:
# - Replace "key = cv2.waitKey(...)" with "key = -1" so later "if key ..." still works but never triggers
s = re.sub(r"^(\s*)key\s*=\s*cv2\.waitKey\(.*\)\s*$", r"\1key = -1  # cv2.waitKey disabled on Colab", s, flags=re.M)
# - Comment out imshow/waitKey calls written in other styles
s = re.sub(r"^(\s*)cv2\.imshow\(.*\)\s*$", r"\1# \g<0>  # disabled on Colab", s, flags=re.M)
s = re.sub(r"^(\s*)cv2\.destroyAllWindows\(\)\s*$", r"\1# cv2.destroyAllWindows()  # disabled on Colab", s, flags=re.M)
# - If code has "if cv2.waitKey(...)" conditions, make them never run (keeps indentation valid)
s = re.sub(r"^(\s*)if\s+cv2\.waitKey\(.*\)\s*&\s*0xFF\s*==\s*ord\('q'\)\s*:\s*$",
           r"\1if False:  # cv2.waitKey disabled on Colab", s, flags=re.M)

# 2) Add VideoWriter saving.
# Insert writer init right after "def draw(...):"
s = re.sub(r"(def draw\(self.*\):\n)", r"\1        os.makedirs('data/out', exist_ok=True)\n        writer = None\n", s, count=1)

# Hook: after a resize of the visualization frame (common patterns: vis or image_combined)
# We'll try both:
def insert_writer_after(pattern):
    global s
    m = re.search(pattern, s, flags=re.M)
    if not m:
        return False
    indent = m.group(1)
    insert = f"""
{indent}# --- Colab save ---
{indent}frame_u8 = frame if frame.dtype==np.uint8 else (np.clip(frame,0,1)*255.0).astype(np.uint8)
{indent}if writer is None:
{indent}    h, w = frame_u8.shape[:2]
{indent}    writer = cv2.VideoWriter('data/out/result.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 25, (w, h))
{indent}writer.write(frame_u8)
{indent}if i % 10 == 0:
{indent}    cv2.imwrite(f"data/out/vis_{'{'}i:04d{'}'}.jpg", frame_u8)
"""
    # Replace using a named variable "frame" right before insert
    # We'll set "frame = <var>" depending on match group
    var = m.group(2)
    insert = insert.replace("frame =", f"frame = {var}")
    s = s[:m.end()] + "\n" + insert + s[m.end():]
    return True

# Pattern 1: "vis = cv2.resize(vis, ...)"  => save vis
ok = insert_writer_after(r"^(\s*)(vis\s*=\s*cv2\.resize\(\s*(vis)\s*,.*\)\s*)$")

# Pattern 2: "image_combined = cv2.resize(image_combined, ...)" => save image_combined
if not ok:
    ok = insert_writer_after(r"^(\s*)(image_combined\s*=\s*cv2\.resize\(\s*(image_combined)\s*,.*\)\s*)$")

# If neither matched, we still keep GUI disabled (won't crash), but no mp4 saving.
if not ok:
    s = "# WARNING: could not auto-insert writer hook; GUI disabled only.\n" + s

p.write_text(s, encoding="utf-8")
print("✅ Patched DanceDemo.py for Colab (GUI disabled; tries to save result.mp4)")
PY


✅ Patched DanceDemo.py for Colab (GUI disabled; tries to save result.mp4)


In [62]:
%cd /content/tp
!mkdir -p data
!cp -v /content/taichi2.mp4 data/taichi2.mp4
!cp -v /content/karate1.mp4 data/karate1.mp4
!ls -lah data/taichi2.mp4 data/karate1.mp4


/content/tp
'/content/taichi2.mp4' -> 'data/taichi2.mp4'
'/content/karate1.mp4' -> 'data/karate1.mp4'
-rw-r--r-- 1 root root 217K Dec 19 21:52 data/karate1.mp4
-rw-r--r-- 1 root root 1.9M Dec 19 21:52 data/taichi2.mp4


In [64]:
%%bash
cd /content/tp

python - <<'PY'
import re, pathlib

p = pathlib.Path("VideoReader.py")
s = p.read_text(encoding="utf-8")

# Find the VideoReader class block
m = re.search(r'^(class\s+VideoReader\s*\(?.*?\)\s*:|class\s+VideoReader\s*:)\s*$',
              s, flags=re.M)
if not m:
    raise SystemExit("Couldn't find: class VideoReader")

start = m.start()

# Find end of class (next top-level 'class ' or 'if __name__' or end of file)
after = s[m.end():]
end_candidates = []
for pat in [r'^\s*class\s+\w+', r'^\s*if\s+__name__\s*==\s*[\'"]__main__[\'"]\s*:']:
    mm = re.search(pat, after, flags=re.M)
    if mm:
        end_candidates.append(m.end() + mm.start())
end = min(end_candidates) if end_candidates else len(s)

class_block = s[start:end]

# If already fixed, do nothing
if re.search(r'^\s+def\s+nbFrame\s*\(', class_block, flags=re.M) and re.search(r'^\s+def\s+readFrame\s*\(\s*self\s*,\s*i\s*=\s*None', class_block, flags=re.M):
    print("VideoReader already has nbFrame() and readFrame(self, i=None). No changes.")
    raise SystemExit

# Append methods at end of class (later defs override earlier ones)
patch = """
    def nbFrame(self):
        # total number of frames in the video
        return int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

    def fps(self):
        fps = float(self.cap.get(cv2.CAP_PROP_FPS))
        return fps if fps > 0 else 25.0

    def readFrame(self, i=None):
        \"""
        If i is None: read next frame (sequential)
        If i is given: jump to frame i then read it
        \"""
        if not self.cap.isOpened():
            return None

        if i is not None:
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, int(i))
            self._current_frame = int(i)

        ret, frame = self.cap.read()
        if not ret or frame is None:
            return None

        self._current_frame += 1
        return frame
"""

# Ensure class ends with newline, then add patch
if not class_block.endswith("\n"):
    class_block += "\n"
class_block += patch

s2 = s[:start] + class_block + s[end:]
p.write_text(s2, encoding="utf-8")
print("Patched VideoReader.py ✅ (added nbFrame, fps, readFrame(i=None))")
PY


Patched VideoReader.py ✅ (added nbFrame, fps, readFrame(i=None))


In [65]:
%cd /content/tp
!rm -rf data/out && mkdir -p data/out
!python DanceDemo.py data/taichi2.mp4 data/karate1.mp4 2
!ls -lah data/out | head


/content/tp
  File "/content/tp/DanceDemo.py", line 92
    break
IndentationError: unexpected indent
total 8.0K
drwxr-xr-x 2 root root 4.0K Dec 19 21:53 .
drwxr-xr-x 4 root root 4.0K Dec 19 21:53 ..


In [66]:
%%bash
cd /content/tp
python - <<'PY'
from VideoReader import VideoReader
vr = VideoReader("data/taichi2.mp4")
print("cap opened:", vr.cap.isOpened())
print("nbFrame:", vr.nbFrame())
print("fps:", vr.fps())
f0 = vr.readFrame(0)
print("frame0 None?", f0 is None, "shape:", None if f0 is None else f0.shape)
PY


cap opened: True
nbFrame: 901
fps: 29.97002963773153
frame0 None? False shape: (720, 960, 3)


In [67]:
%%bash
cd /content/tp

python - <<'PY'
import pathlib, re

p = pathlib.Path("DanceDemo.py")
s = p.read_text(encoding="utf-8")

# Ensure output dir exists somewhere in draw()
# We'll:
#  - remove/disable cv2.imshow + cv2.waitKey blocks
#  - ensure we convert vis to uint8 before writing
#  - write frames to data/out and also an mp4

# 1) Comment out any cv2.imshow lines
s = re.sub(r'^\s*cv2\.imshow\(.*\)\s*$', r'# \g<0>  # disabled on Colab', s, flags=re.M)
# 2) Comment out any cv2.waitKey lines and any 'if cv2.waitKey...' blocks
s = re.sub(r'^\s*key\s*=\s*cv2\.waitKey\(.*\)\s*$', r'# \g<0>  # disabled on Colab', s, flags=re.M)
s = re.sub(r'^\s*if\s+cv2\.waitKey\(.*\):\s*$', r'# \g<0>  # disabled on Colab', s, flags=re.M)

# Add helper just once
if "def _to_u8(" not in s:
    helper = """
def _to_u8(img):
    \"\"\"Ensure uint8 BGR image for cv2.imwrite / VideoWriter.\"\"\"
    import numpy as np
    if img is None:
        return None
    if img.dtype != np.uint8:
        img = np.clip(img, 0.0, 1.0) if img.max() <= 1.5 else img
        if img.max() <= 1.5:
            img = (img * 255.0)
        img = img.astype(np.uint8)
    return img
"""
    # Put helper near top (after imports)
    s = re.sub(r'(\nfrom\s+.*\n)', r'\1' + helper + "\n", s, count=1)

# Make sure draw() writes frames & mp4
# We'll inject minimal writer logic inside draw() after 'def draw' line.
if "VideoWriter" not in s or "data/out/result.mp4" not in s:
    s = re.sub(
        r'(def\s+draw\s*\(\s*self\s*\)\s*:\s*\n)',
        r'\1'
        r'        import os, cv2\n'
        r'        os.makedirs("data/out", exist_ok=True)\n'
        r'        writer = None\n'
        r'        out_mp4 = "data/out/result.mp4"\n'
        r'        print("DanceDemo: saving output to", out_mp4)\n',
        s,
        count=1
    )

# Replace any writer.write(vis) with uint8 write
s = re.sub(r'writer\.write\(\s*vis\s*\)', r'writer.write(_to_u8(vis))', s)

# If file writes exist but vis_u8 is sometimes missing, make sure vis_u8 defined right after vis created
# (common pattern: vis = np.concatenate(...))
s = re.sub(
    r'(\n\s*vis\s*=\s*.*\n)',
    r'\1' + r'            vis_u8 = _to_u8(vis)\n',
    s
)

# Ensure cv2.imwrite uses vis_u8
s = re.sub(r'cv2\.imwrite\(([^,]+),\s*vis\s*\)', r'cv2.imwrite(\1, vis_u8)', s)

# Ensure VideoWriter is created once we know vis size
# If "writer is None" block isn't present, insert after first vis_u8 creation
if "if writer is None" not in s:
    s = re.sub(
        r'(vis_u8\s*=\s*_to_u8\(vis\)\s*\n)',
        r'\1'
        r'            if writer is None:\n'
        r'                h, w = vis_u8.shape[:2]\n'
        r'                fps = 25\n'
        r'                try:\n'
        r'                    fps = int(getattr(self.source, "fps", lambda: 25)())\n'
        r'                except Exception:\n'
        r'                    fps = 25\n'
        r'                fourcc = cv2.VideoWriter_fourcc(*"mp4v")\n'
        r'                writer = cv2.VideoWriter(out_mp4, fourcc, fps, (w, h))\n',
        s,
        count=1
    )

# Ensure we release writer at end of draw()
if "writer.release()" not in s:
    s = re.sub(
        r'(\n\s*def\s+_usage\s*\(\)\s*:)',
        r'\n        if writer is not None:\n'
        r'            writer.release()\n'
        r'            print("DanceDemo: wrote", out_mp4)\n'
        r'\1',
        s,
        count=1
    )

p.write_text(s, encoding="utf-8")
print("Patched DanceDemo.py ✅ (no imshow/waitKey; saves frames + mp4 to data/out)")
PY


Patched DanceDemo.py ✅ (no imshow/waitKey; saves frames + mp4 to data/out)


In [68]:
%%bash
cd /content/tp
python - <<'PY'
import os, glob
out="data/out/result.mp4"
print("mp4 exists:", os.path.exists(out), "size:", os.path.getsize(out) if os.path.exists(out) else None)
print("jpg frames:", len(glob.glob("data/out/vis_*.jpg")))
PY


mp4 exists: False size: None
jpg frames: 0


In [69]:
%%bash
cd /content/tp
python - <<'PY'
import re, pathlib

p = pathlib.Path("VideoReader.py")
s = p.read_text(encoding="utf-8")

# Replace any existing readFrame(...) definition with an indexed-friendly version
pattern = r"def\s+readFrame\s*\(self[^\)]*\)\s*:\s*\n(?:[ \t].*\n)+"
replacement = """def readFrame(self, i=None):
    \"\"\"Read a frame.
    If i is None: read next frame (sequential).
    If i is not None: seek to frame i then read.
    \"\"\"
    if not self.cap.isOpened():
        return None

    if i is not None:
        # seek to i-th frame
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, int(i))

    ret, frame = self.cap.read()
    if not ret or frame is None:
        return None

    return frame
"""

s2, n = re.subn(pattern, replacement, s, count=1)
if n == 0:
    # if regex didn't match, just append a correct version (rare case)
    s2 = s + "\n\n" + replacement

p.write_text(s2, encoding="utf-8")
print("✅ Patched VideoReader.readFrame(self, i=None)")
PY


✅ Patched VideoReader.readFrame(self, i=None)


In [71]:
%%bash
cd /content/tp
python - <<'PY'
import re
from pathlib import Path

p = Path("VideoReader.py")
lines = p.read_text(encoding="utf-8").splitlines(True)

re_def = re.compile(r'^(\s*)def\s+readFrame\s*\(')

out = []
i = 0

while i < len(lines):
    m = re_def.match(lines[i])
    if not m:
        out.append(lines[i])
        i += 1
        continue

    indent = m.group(1)                 # indentation of "def readFrame"
    body_indent = indent + "    "       # indentation of the function body

    # skip old readFrame block
    j = i + 1
    while j < len(lines):
        # next method at same indent
        if re.match(r'^%sdef\s+\w+' % re.escape(indent), lines[j]):
            break
        # dedent out of class/method
        if lines[j].strip() and (len(lines[j]) - len(lines[j].lstrip())) < len(indent):
            break
        j += 1

    # insert corrected method
    new = [
        f"{indent}def readFrame(self, i=None):\n",
        f"{body_indent}\"\"\"Read a frame.\n",
        f"{body_indent}If i is None: read next frame (sequential).\n",
        f"{body_indent}If i is not None: seek to frame i then read.\n",
        f"{body_indent}\"\"\"\n",
        f"{body_indent}if not self.cap.isOpened():\n",
        f"{body_indent}    return None\n",
        "\n",
        f"{body_indent}if i is not None:\n",
        f"{body_indent}    self.cap.set(cv2.CAP_PROP_POS_FRAMES, int(i))\n",
        "\n",
        f"{body_indent}ret, frame = self.cap.read()\n",
        f"{body_indent}if not ret or frame is None:\n",
        f"{body_indent}    return None\n",
        "\n",
        f"{body_indent}return frame\n",
    ]

    out.extend(new)
    i = j

p.write_text("".join(out), encoding="utf-8")
print("✅ Fixed: readFrame indentation + supports readFrame(i)")
PY


✅ Fixed: readFrame indentation + supports readFrame(i)


In [72]:
%%bash
cd /content/tp
python - <<'PY'
from VideoReader import VideoReader
vr = VideoReader("data/taichi2.mp4")
f0 = vr.readFrame(0)
print("frame0 is None?", f0 is None, "shape:", None if f0 is None else f0.shape)
PY


frame0 is None? False shape: (720, 960, 3)


In [73]:
%%bash
cd /content/tp
cat > DanceDemo.py <<'PY'
import os, sys
import numpy as np
import cv2

from VideoReader import VideoReader
from VideoSkeleton import VideoSkeleton

from GenNearest import GenNearest
from GenVanillaNN import GenVanillaNN
from GenGAN import GenGAN


class DanceDemo:
    def __init__(self, filename_src, filename_tgt, typeOfGen=2):
        self.filename_src = filename_src
        self.filename_tgt = filename_tgt
        self.typeOfGen = int(typeOfGen)

        self.source = VideoReader(filename_src)
        self.target = VideoSkeleton(filename_tgt)  # loads .pkl if exists, else computes

        if self.typeOfGen == 1:
            print("Generator: Nearest neighbor")
            self.generator = GenNearest(self.target)
        elif self.typeOfGen == 2:
            print("Generator: GenVanillaNN (ske26 -> image)")
            self.generator = GenVanillaNN(self.target, loadFromFile=True, optSkeOrImage=1)
        elif self.typeOfGen == 3:
            print("Generator: GenVanillaNN (stick -> image)")
            self.generator = GenVanillaNN(self.target, loadFromFile=True, optSkeOrImage=2)
        elif self.typeOfGen == 4:
            print("Generator: GenGAN (stick -> image)")
            self.generator = GenGAN(self.target, loadFromFile=True)
        else:
            raise ValueError("Unknown gen_type")

    def _to_u8_bgr(self, img):
        """
        Accepts either:
          - uint8 BGR
          - float BGR in [0,1]
        Returns uint8 BGR.
        """
        if img is None:
            return None
        if img.dtype == np.uint8:
            return img
        # assume float
        m = float(np.max(img)) if img.size else 1.0
        if m <= 1.5:
            img = (img * 255.0)
        img = np.clip(img, 0, 255).astype(np.uint8)
        return img

    def draw(self, out_dir="data/out", out_name="result.mp4", max_frames=None, save_jpg_every=0):
        os.makedirs(out_dir, exist_ok=True)

        nb_src = self.source.nbFrame()
        nb_tgt = self.target.skeCount()
        n = min(nb_src, nb_tgt)
        if max_frames is not None:
            n = min(n, int(max_frames))

        fps = self.source.fps()
        if fps is None or fps <= 1e-6:
            fps = 30.0

        writer = None
        out_path = os.path.join(out_dir, out_name)
        print("DanceDemo: saving output to", out_path)
        print("DanceDemo: frames:", n, "fps:", fps)

        for i in range(n):
            frame = self.source.readFrame(i)        # seek+read
            if frame is None:
                print("DanceDemo: source frame None at", i, "=> stop")
                break

            ske = self.target.ske[i]

            # generate image from skeleton
            gen = self.generator.generate(ske)
            gen_u8 = self._to_u8_bgr(gen)
            if gen_u8 is None:
                print("DanceDemo: generated None at", i, "=> stop")
                break

            # choose a nice display size
            H = 256
            W = 256
            src_disp = cv2.resize(frame, (W, H))

            gen_disp = cv2.resize(gen_u8, (W, H))

            stick = np.ones((H, W, 3), dtype=np.uint8) * 255
            # draw skeleton onto stick canvas (works in your project)
            ske.draw(stick)

            vis = np.concatenate([src_disp, gen_disp, stick], axis=1)  # uint8

            if writer is None:
                fourcc = cv2.VideoWriter_fourcc(*"mp4v")
                writer = cv2.VideoWriter(out_path, fourcc, fps, (vis.shape[1], vis.shape[0]))
                if not writer.isOpened():
                    raise RuntimeError("Could not open VideoWriter. Try changing fourcc to 'avc1' or 'XVID'.")

            writer.write(vis)

            if save_jpg_every and (i % int(save_jpg_every) == 0):
                cv2.imwrite(os.path.join(out_dir, f"vis_{i:04d}.jpg"), vis)

            if (i % 25) == 0:
                print(f"  wrote frame {i}/{n}")

        if writer is not None:
            writer.release()

        print("DanceDemo: wrote", out_path)


def _usage():
    print("Usage:")
    print("  python DanceDemo.py <source_video.mp4> <target_video.mp4> [gen_type]")
    print("")
    print("gen_type:")
    print("  1 = Nearest neighbor")
    print("  2 = Vanilla NN (ske26 -> image)")
    print("  3 = Vanilla NN (stick -> image)")
    print("  4 = GAN (stick -> image)")
    print("")
    print("Examples:")
    print("  python DanceDemo.py data/taichi2.mp4 data/karate1.mp4 2")
    print("  python DanceDemo.py data/taichi2.mp4 data/karate1.mp4 3")


if __name__ == "__main__":
    if len(sys.argv) < 3:
        _usage()
        sys.exit(0)

    src = sys.argv[1]
    tgt = sys.argv[2]
    gen_type = int(sys.argv[3]) if len(sys.argv) >= 4 else 2

    demo = DanceDemo(src, tgt, typeOfGen=gen_type)
    demo.draw(out_dir="data/out", out_name="result.mp4", save_jpg_every=0)
PY
echo "✅ Patched DanceDemo.py (headless export, no imshow/waitKey)"


✅ Patched DanceDemo.py (headless export, no imshow/waitKey)


In [74]:
%%bash
cd /content/tp
python - <<'PY'
import re

p="DanceDemo.py"
s=open(p,"r",encoding="utf-8").read()

# 1) Remove the GenNearest import line (if present)
s = re.sub(r'^\s*from\s+GenNearest\s+import\s+GenNearest\s*\n', '', s, flags=re.M)

# 2) Remove the whole gen_type==1 branch
s = re.sub(
    r'\n\s*if\s+self\.typeOfGen\s*==\s*1:\n(?:.*\n)*?\s*elif\s+self\.typeOfGen\s*==\s*2:',
    '\n        if self.typeOfGen == 2:',
    s,
    flags=re.M
)

open(p,"w",encoding="utf-8").write(s)
print("✅ Patched DanceDemo.py: removed GenNearest dependency (gen_type 1).")
PY


✅ Patched DanceDemo.py: removed GenNearest dependency (gen_type 1).


In [76]:
%%bash
cd /content/tp
python - <<'PY'
import re, pathlib

p = pathlib.Path("DanceDemo.py")
s = p.read_text(encoding="utf-8")

# 1) Ensure we build skeletons for source video too
# Insert after: self.source = VideoReader(filename_src)
if "self.sourceSke" not in s:
    s = re.sub(
        r"(self\.source\s*=\s*VideoReader\(filename_src\)\s*\n)",
        r"\1        self.sourceSke = VideoSkeleton(filename_src)  # NEW: source skeletons (taichi motion)\n",
        s,
        count=1
    )

# 2) Replace pose input from target skeleton -> source skeleton
s = s.replace("ske = self.target.ske[i]", "ske = self.sourceSke.ske[i]")

# 3) Fix n calculation: use skeleton counts, not raw video frames
# Replace block (works for your current file structure)
s = re.sub(
    r"nb_src\s*=\s*self\.source\.nbFrame\(\)\s*\n\s*nb_tgt\s*=\s*self\.target\.skeCount\(\)\s*\n\s*n\s*=\s*min\(nb_src,\s*nb_tgt\)\s*\n",
    "nb_src_ske = self.sourceSke.skeCount()\n        nb_tgt = self.target.skeCount()\n        n = min(nb_src_ske, nb_tgt)\n",
    s,
    count=1
)

p.write_text(s, encoding="utf-8")
print("✅ Patched DanceDemo.py:")
print(" - added self.sourceSke = VideoSkeleton(filename_src) (if missing)")
print(" - now uses ske = self.sourceSke.ske[i]")
print(" - n now uses sourceSke.skeCount()")
PY


✅ Patched DanceDemo.py:
 - added self.sourceSke = VideoSkeleton(filename_src) (if missing)
 - now uses ske = self.sourceSke.ske[i]
 - n now uses sourceSke.skeCount()


In [77]:
%%bash
cd /content/tp
grep -n "sourceSke" -n DanceDemo.py | head
grep -n "ske = self\." DanceDemo.py | head


18:        self.sourceSke = VideoSkeleton(filename_src)  # NEW: source skeletons (taichi motion)
53:        nb_src_ske = self.sourceSke.skeCount()
74:            ske = self.sourceSke.ske[i]
53:        nb_src_ske = self.sourceSke.skeCount()
74:            ske = self.sourceSke.ske[i]


In [82]:
%%bash
set -e
cd /content/tp

# confirm mp4s exist
ls -lah data/taichi2.mp4 data/karate1.mp4

# remove corrupted karate cache
rm -rf data/karate1 data/karate1.pkl

# recompute karate skeleton+frames cleanly
python - <<'PY'
from VideoSkeleton import VideoSkeleton
VideoSkeleton("data/karate1.mp4")   # will compute and save data/karate1.pkl
print("✅ Recomputed karate1.pkl")
PY

# verify it's NOT zero anymore
python - <<'PY'
from VideoSkeleton import VideoSkeleton
v = VideoSkeleton("data/karate1.mp4")
print("karate skeletons:", v.skeCount(), "images:", len(v.im))
PY


-rw-r--r-- 1 root root 217K Dec 19 21:52 data/karate1.mp4
-rw-r--r-- 1 root root 1.9M Dec 19 21:52 data/taichi2.mp4
directory: data/karate1
create directory: data/karate1
===== compute: data/karate1.mp4
read: data/karate1.mp4 #frame=192
video shape=(360, 640, 3)
new_video_width=256 new_video_height=144
 ske_width_crop=256 ske_height_crop=144
frame 0/192   filename=karate1/image0.jpg  save=data/karate1/image0.jpg sizeof=88
frame 10/192   filename=karate1/image10.jpg  save=data/karate1/image10.jpg sizeof=88
frame 20/192   filename=karate1/image20.jpg  save=data/karate1/image20.jpg sizeof=88
frame 30/192   filename=karate1/image30.jpg  save=data/karate1/image30.jpg sizeof=88
frame 40/192   filename=karate1/image40.jpg  save=data/karate1/image40.jpg sizeof=120
frame 50/192   filename=karate1/image50.jpg  save=data/karate1/image50.jpg sizeof=120
frame 60/192   filename=karate1/image60.jpg  save=data/karate1/image60.jpg sizeof=120
frame 70/192   filename=karate1/image70.jpg  save=data/karate

2025-12-19 22:12:59.276104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766182379.310730   18421 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766182379.320122   18421 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 22:12:59.359992: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1766182388.200535   18489 inference_feedback_manager.cc:1

In [92]:
%%bash
set -e
cd /content/tp

python - <<'PY'
from pathlib import Path
import re

p = Path("GenVanillaNN.py")
s = p.read_text(encoding="utf-8")

# --- 1) Save as state_dict (not full model) ---
# Replace: torch.save(self.netG, self.filename)  -> torch.save(self.netG.state_dict(), self.filename)
s, n_save = re.subn(
    r"torch\.save\(\s*self\.netG\s*,\s*self\.filename\s*\)",
    "torch.save(self.netG.state_dict(), self.filename)",
    s
)

# --- 2) Load robustly (state_dict OR old full-model) ---
# Replace the simple torch.load line with robust block
pattern_load = r"^\s*self\.netG\s*=\s*torch\.load\(\s*self\.filename\s*,\s*map_location=self\.device\s*\)\s*$"
repl_load = """\
        ckpt = torch.load(self.filename, map_location=self.device, weights_only=False)
        # New format: state_dict
        if isinstance(ckpt, dict):
            self.netG.load_state_dict(ckpt)
        else:
            # Old format: full model object (may require same __main__ context)
            self.netG = ckpt.to(self.device)
"""
s, n_load = re.subn(pattern_load, repl_load, s, flags=re.M)

# --- 3) Fix tensor2image float64 -> uint8 before cv2.cvtColor (prevents CV_64F crash) ---
s, n_rgb = re.subn(
    r"cv2\.cvtColor\(\s*np\.array\(denorm\)\s*,\s*cv2\.COLOR_RGB2BGR\s*\)",
    "cv2.cvtColor((np.clip(np.array(denorm),0,1)*255).astype(np.uint8), cv2.COLOR_RGB2BGR)",
    s
)

p.write_text(s, encoding="utf-8")
print(f"✅ Patched GenVanillaNN.py | save_state_dict: {n_save} | load_patch: {n_load} | tensor2image_fix: {n_rgb}")
PY

python -m py_compile /content/tp/GenVanillaNN.py
echo "✅ GenVanillaNN.py compiles"


✅ Patched GenVanillaNN.py | save_state_dict: 1 | load_patch: 1 | tensor2image_fix: 0
✅ GenVanillaNN.py compiles


In [93]:
%%bash
set -e
cd /content/tp

python GenVanillaNN.py data/karate1.mp4

ls -lah data/Dance | head -n 20
ls -lah data/out | head -n 20


GenVanillaNN: Current Working Directory= /content/tp
GenVanillaNN: Filename= data/karate1.mp4
directory: data/karate1
===== read precompute: data/karate1.mp4
VideoSkeleton::load: data/karate1.pkl #skeleton=(17,) #image=(17,)
Sequential(
  (0): ConvTranspose2d(26, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine

2025-12-19 22:41:54.992844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766184115.026574   25489 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766184115.035210   25489 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 22:41:55.064757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1766184123.785176   25577 inference_feedback_manager.cc:1

In [94]:
%%bash
set -e
cd /content/tp
rm -rf data/out && mkdir -p data/out

python DanceDemo.py data/taichi2.mp4 data/karate1.mp4 2

ls -lah data/out


directory: data/taichi2
===== read precompute: data/taichi2.mp4
VideoSkeleton::load: data/taichi2.pkl #skeleton=(91,) #image=(91,)
directory: data/karate1
===== read precompute: data/karate1.mp4
VideoSkeleton::load: data/karate1.pkl #skeleton=(17,) #image=(17,)
Generator: GenVanillaNN (ske26 -> image)
Sequential(
  (0): ConvTranspose2d(26, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), paddi

2025-12-19 22:43:14.180324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766184194.224390   25858 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766184194.236794   25858 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 22:43:14.276399: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1766184202.196135   25922 inference_feedback_manager.cc:1

In [104]:
%%bash
set -e
cd /content/tp

python - <<'PY'
from pathlib import Path
import re

p = Path("GenVanillaNN.py")
s = p.read_text(encoding="utf-8")

# Comment out GUI calls safely (keeps indentation)
s = re.sub(r'(?m)^(\s*)cv2\.imshow\(.*\)\s*$', r'\1# cv2.imshow(...)  # disabled on Colab', s)
s = re.sub(r'(?m)^(\s*)key\s*=\s*cv2\.waitKey\(.*\)\s*$', r'\1# key = cv2.waitKey(...)  # disabled on Colab', s)
s = re.sub(r'(?m)^(\s*)cv2\.waitKey\(.*\)\s*$', r'\1# cv2.waitKey(...)  # disabled on Colab', s)
s = re.sub(r'(?m)^(\s*)cv2\.destroyAllWindows\(\)\s*$', r'\1# cv2.destroyAllWindows()  # disabled on Colab', s)

# If there is logic like: if key == 27: break  -> comment it (prevents undefined key)
s = re.sub(r'(?m)^(\s*)if\s+key\s*==\s*\d+\s*:\s*$', r'\1# if key == ...:  # disabled on Colab', s)
s = re.sub(r'(?m)^(\s*)break\s*$', r'\1# break  # disabled on Colab', s)

p.write_text(s, encoding="utf-8")
print("✅ Patched GenVanillaNN.py to disable OpenCV GUI calls.")
PY

python -m py_compile /content/tp/GenVanillaNN.py
echo "✅ still compiles"


✅ Patched GenVanillaNN.py to disable OpenCV GUI calls.
✅ still compiles


In [103]:
%%bash
set -e
cd /content/tp

python GenVanillaNN.py data/karate1.mp4

ls -lah data/Dance | tail -n 30


GenVanillaNN: Current Working Directory= /content/tp
GenVanillaNN: Filename= data/karate1.mp4
directory: data/karate1
===== read precompute: data/karate1.mp4
VideoSkeleton::load: data/karate1.pkl #skeleton=(17,) #image=(17,)
GenNNSkeImToImage(
  (enc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

2025-12-19 23:01:52.537673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766185312.593127   30341 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766185312.609301   30341 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 23:01:52.653470: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1766185321.362501   30434 inference_feedback_manager.cc:1

CalledProcessError: Command 'b'set -e\ncd /content/tp\n\npython GenVanillaNN.py data/karate1.mp4\n\nls -lah data/Dance | tail -n 30\n'' returned non-zero exit status 1.

In [105]:
%%bash
set -e
cd /content/tp
rm -rf data/out && mkdir -p data/out

python DanceDemo.py data/taichi2.mp4 data/karate1.mp4 2

ls -lah data/out


directory: data/taichi2
===== read precompute: data/taichi2.mp4
VideoSkeleton::load: data/taichi2.pkl #skeleton=(91,) #image=(91,)
directory: data/karate1
===== read precompute: data/karate1.mp4
VideoSkeleton::load: data/karate1.pkl #skeleton=(17,) #image=(17,)
Generator: GenVanillaNN (ske26 -> image)
Sequential(
  (0): ConvTranspose2d(26, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), paddi

2025-12-19 23:06:44.174952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766185604.204965   31556 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766185604.213043   31556 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 23:06:44.240417: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1766185612.836449   31624 inference_feedback_manager.cc:1

In [111]:
%%bash
set -e
cd /content/tp

# show where waitKey is
echo "---- BEFORE ----"
grep -nE "imshow|waitKey|destroyAllWindows" GenGAN.py || true

python - <<'PY'
from pathlib import Path
import re

p = Path("GenGAN.py")
s = p.read_text(encoding="utf-8")

# comment out ANY line that contains these calls (even if assigned to a variable)
s = re.sub(r'(?m)^(\s*.*cv2\.imshow\([^\n]*\).*)$', r'# \1  # disabled on Colab', s)
s = re.sub(r'(?m)^(\s*.*cv2\.waitKey\([^\n]*\).*)$', r'# \1  # disabled on Colab', s)
s = re.sub(r'(?m)^(\s*.*cv2\.destroyAllWindows\(\).*)$', r'# \1  # disabled on Colab', s)

p.write_text(s, encoding="utf-8")
print("✅ Patched GenGAN.py: all imshow/waitKey/destroyAllWindows lines commented.")
PY

echo "---- AFTER ----"
grep -nE "imshow|waitKey|destroyAllWindows" GenGAN.py || true

python -m py_compile GenGAN.py
echo "✅ GenGAN.py compiles"


---- BEFORE ----
198:        # cv2.imshow(...)  # disabled on Colab
199:        key = cv2.waitKey(-1)
✅ Patched GenGAN.py: all imshow/waitKey/destroyAllWindows lines commented.
---- AFTER ----
198:#         # cv2.imshow(...)  # disabled on Colab  # disabled on Colab
199:#         key = cv2.waitKey(-1)  # disabled on Colab
✅ GenGAN.py compiles


In [110]:
%%bash
set -e
cd /content/tp

python GenGAN.py data/karate1.mp4

ls -lah data/Dance | egrep "DanceGenGAN_(G|D)\.pth" || true


GenGAN: Current Working Directory= /content/tp
GenGAN: Filename= data/karate1.mp4
directory: data/karate1
===== read precompute: data/karate1.mp4
VideoSkeleton::load: data/karate1.pkl #skeleton=(17,) #image=(17,)
GenNNSkeImToImage(
  (enc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Leak

2025-12-19 23:42:59.431818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766187779.463393   40239 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766187779.471499   40239 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 23:42:59.499123: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1766187787.456729   40331 inference_feedback_manager.cc:1

CalledProcessError: Command 'b'set -e\ncd /content/tp\n\npython GenGAN.py data/karate1.mp4\n\nls -lah data/Dance | egrep "DanceGenGAN_(G|D)\\.pth" || true\n'' returned non-zero exit status 1.

In [112]:
%%bash
set -e
cd /content/tp

rm -rf data/out && mkdir -p data/out
python DanceDemo.py data/taichi2.mp4 data/karate1.mp4 2

ls -lah data/out | head -n 50


directory: data/taichi2
===== read precompute: data/taichi2.mp4
VideoSkeleton::load: data/taichi2.pkl #skeleton=(91,) #image=(91,)
directory: data/karate1
===== read precompute: data/karate1.mp4
VideoSkeleton::load: data/karate1.pkl #skeleton=(17,) #image=(17,)
Generator: GenVanillaNN (ske26 -> image)
Sequential(
  (0): ConvTranspose2d(26, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), paddi

2025-12-19 23:47:17.607579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766188037.649364   41328 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766188037.661622   41328 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 23:47:17.700994: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1766188044.844160   41388 inference_feedback_manager.cc:1